# O objetivo da camada gold é:

- Contém dados preparados e otimizados para análise e consumo pelas áreas de negócio. 
- Os dados são agregados e formatados para uso em dashboards, relatórios e outras ferramentas de análise. 
- Esta camada normalmente possui tabelas já prontas para consultas, facilitando a tomada de decisões.
- Apresenta dados analíticos e transacionais consolidados, prontos para serem utilizados pelos usuários finais.
- Região Desconhecida deve ser tratada.
- Criação de tabela consumo de energia com informações de cliente e clima. 

In [3]:
import pandas as pd
import duckdb

import mlflow
from mlflow.tracking import MlflowClient

# Extração

- Tabela Consumo
- Tabela Clima
- Tabela Clientes

In [4]:
# Caminho do banco
db_path = "../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Carrega os dados da camada bronze
clientes_df = con.execute("SELECT * FROM silver.clientes").df()
consumo_df = con.execute("SELECT * FROM silver.consumo").df()
clima_df = con.execute("SELECT * FROM silver.clima").df()

In [5]:
# Cria schema 'silver' se não existir
con.execute("CREATE SCHEMA IF NOT EXISTS gold")

# Transformação

In [6]:
cliente_consumo_df = pd.merge(consumo_df, clientes_df, on='client_id', how='inner')

clientes_desconhecidos_df = cliente_consumo_df[cliente_consumo_df['region'] == 'Desconhecida'].copy()
clientes_desconhecidos_df = clientes_desconhecidos_df.drop('region', axis=1)


clientes_conhecidos_df = cliente_consumo_df[cliente_consumo_df['region'] != 'Desconhecida']

In [7]:
# Define o caminho onde os dados da run serão armazenados
mlflow.set_tracking_uri("../../mlruns")

# Inicializa o cliente MLflow
client = MlflowClient()

# Nome do experimento
experiment_name = "modelo_classificacao_pipeline"
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    %run ../region_classificador/train.ipynb
    %run ../region_classificador/predict.ipynb

# Lista os runs do experimento
runs = client.search_runs(experiment_ids=[experiment.experiment_id], order_by=["start_time DESC"])

# Seleciona o run mais recente
latest_run = runs[0]
run_id = latest_run.info.run_id

# Recupera as métricas
metrics = latest_run.data.metrics

# Exibe F1-score
f1_score = metrics.get("f1_score")

# Se o modelo não atingir a métrica mínima de F1-score, não executa a classificação e os dados são excluídos
if f1_score >= 0.8:
    classificador_output_df = con.execute("SELECT * FROM output.region_classificador").df()
    clientes_desconhecidos_df = clientes_desconhecidos_df.merge(
        classificador_output_df[['client_id', 'region']],
        on='client_id',
        how='inner'
    )

    clientes_df = pd.concat([clientes_conhecidos_df, clientes_desconhecidos_df])
else:
    print(f"Modelo não atingiu a métrica mínima de F1-score: {f1_score}. Dados de clientes desconhecidos serão excluídos.")
    clientes_df = clientes_conhecidos_df


Modelo não atingiu a métrica mínima de F1-score: 0.2553823836744729. Dados de clientes desconhecidos serão excluídos.


In [8]:
consumo_geral = (
    cliente_consumo_df
    .merge(
        clima_df,
        on=['region', 'date'], 
        how='inner'
    )
)

consumo_geral = consumo_geral.sort_values(by=['region', 'date'])


# Carregamento

In [9]:
consumo_geral = consumo_geral[['date', 'client_id', 'region', 'temperature', 'humidity', 'consumption_kwh']]

In [10]:
# Cria tabelas dentro do schema gold
con.execute("""
CREATE TABLE IF NOT EXISTS gold.consumo_geral (
    date DATE,
    client_id VARCHAR,
    region VARCHAR,
    temperature DOUBLE,
    humidity DOUBLE,
    consumption_kwh DOUBLE
)
""")

In [11]:
# Limpa dados se as tabelas já existirem
con.execute("DELETE FROM gold.consumo_geral")

In [12]:
# Registra como tabelas temporárias
con.register("consumo_geral", consumo_geral)

In [13]:
# Insere os dados nas tabelas gold
con.execute("INSERT INTO gold.consumo_geral SELECT * FROM consumo_geral")

In [14]:
con.close()
print("✔ Dados inseridos com sucesso na camada gold do DuckDB.")

✔ Dados inseridos com sucesso na camada gold do DuckDB.
